Установка необходимых библиотек

In [73]:
!pip install gigachain gigachain-community GigaChat unstructured[all-docs] tools faiss-cpu sentence-transformers fitz frontend pymupdf PyPDF2 fitz langchain_community gigachain colpali-engine transformers torch torchvision python-docx huggingface-hub --quiet

Импорты

In [23]:
import os
import json
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from PyPDF2 import PdfReader

import fitz
from pdf2image import convert_from_path
from PIL import Image

Извлечение текста из PDF файлов

In [87]:
def extract_pdf_text_to_folder(input_folder, output_folder):
    """
    Извлекает текст из всех PDF-файлов в указанной папке и сохраняет в текстовые файлы.
    """
    # Убедимся, что выходная папка существует
    os.makedirs(output_folder, exist_ok=True)

    # Получаем список PDF-файлов в папке
    pdf_files = [f for f in os.listdir(input_folder) if f.endswith('.pdf')]

    if not pdf_files:
        print("Нет PDF-файлов в указанной папке.")
        return

    # Проходим по каждому PDF-файлу
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_folder, pdf_file)
        output_file = os.path.join(output_folder, os.path.splitext(pdf_file)[0] + ".txt")

        try:
            # Открываем PDF-файл
            reader = PdfReader(pdf_path)

            # Создаём или очищаем файл для записи
            with open(output_file, "w", encoding="utf-8") as f:
                # Проходимся по страницам PDF
                for page_num, page in enumerate(reader.pages, start=1):
                    text = page.extract_text()
                    if text:  # Проверяем, есть ли текст на странице
                        f.write(f"Page {page_num}\n")  # Добавляем метку страницы
                        # result = get_embedding_from_gigachain(text)
                        f.write(text.strip() + "\n\n")  # Добавляем текст страницы
                    else:
                        f.write(f"Page {page_num}\n")  # Добавляем метку пустой страницы
                        f.write("Страница пуста или текст не удалось извлечь.\n\n")

            print(f"Текст извлечен из {pdf_file} и сохранен в {output_file}")

        except Exception as e:
            print(f"Ошибка при обработке {pdf_file}: {e}")

# Пример использования
input_folder = "pdfs"  # Папка с PDF-файлами
output_folder = "extracted_texts"  # Папка для сохранения текстов
extract_pdf_text_to_folder(input_folder, output_folder)


Текст извлечен из sr_ru_annual_report_pages_nornik_2022.pdf и сохранен в extracted_texts/sr_ru_annual_report_pages_nornik_2022.txt
Текст извлечен из Норникель про корп культуру.pdf и сохранен в extracted_texts/Норникель про корп культуру.txt
Текст извлечен из responsible_supply_chain_report_rus.pdf и сохранен в extracted_texts/responsible_supply_chain_report_rus.txt


Текст извлечен из NN_CSO2021_RUS_03.03.2023.pdf и сохранен в extracted_texts/NN_CSO2021_RUS_03.03.2023.txt
Текст извлечен из 2_5366183129474161642.pdf и сохранен в extracted_texts/2_5366183129474161642.txt
Текст извлечен из Норникель_Внутрення_цена_на_углерод.pdf и сохранен в extracted_texts/Норникель_Внутрення_цена_на_углерод.txt
Текст извлечен из digital_production_5.pdf и сохранен в extracted_texts/digital_production_5.txt
Текст извлечен из 2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.pdf и сохранен в extracted_texts/2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.txt
Текст извлечен из ММК 2024.pdf и сохранен в extracted_texts/ММК 2024.txt
Текст извлечен из McKinsey_Next Big Arenas_2024 (213 pgs).pdf и сохранен в extracted_texts/McKinsey_Next Big Arenas_2024 (213 pgs).txt
Текст извлечен из NN_AR_2021_Book_RUS_26.09.22.pdf и сохранен в extracted_texts/NN_AR_2021_Book_RUS_26.09.22.txt
Текст извлечен из Доклад, уголь часть 1.pdf и сохранен в extracted_texts/Доклад, уголь часть 1

Попытка починить неисправные pdf файлы, если конечно такие присутствуют

In [83]:
def extract_text_from_pdf(pdf_path):
    """Извлекает текст из PDF с использованием разных методов."""
    text = ""

    try:
        # Попробуем использовать PyPDF2 для извлечения текста
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text() or ""
    except Exception as e:
        print(f"Ошибка при извлечении текста через PyPDF2: {e}")

    if text.strip():
        return text

    try:
        # Если PyPDF2 не смог извлечь текст, попробуем использовать pdf2image и Tesseract
        images = convert_from_path(pdf_path, 300)  # Конвертируем PDF в изображения
        text = ""
        for img in images:
            text += pytesseract.image_to_string(img)  # Преобразуем изображение в текст
    except Exception as e:
        print(f"Ошибка при извлечении текста через Tesseract: {e}")

    if text.strip():
        return text

    try:
        # Используем PyMuPDF (fitz) для извлечения текста
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text("text")  # Получаем текст в обычном формате
    except Exception as e:
        print(f"Ошибка при извлечении текста через PyMuPDF: {e}")

    return text.strip() if text.strip() else None

def clean_text(text):
    """Очистка текста от лишних пробелов и символов."""
    text = text.replace('\n', ' ')  # Заменяем переводы строк на пробелы
    text = ' '.join(text.split())  # Убираем лишние пробелы

    return result

def process_pdf(pdf_path):
    """Основная функция для обработки PDF и восстановления текста."""
    text = extract_text_from_pdf(pdf_path)
    if text:
        cleaned_text = clean_text(text)
        output_file = os.path.splitext(pdf_path)[0] + "_extracted.txt"
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(cleaned_text)
        print(f"Текст успешно извлечен и сохранен в файл: {output_file}")
    else:
        print("Не удалось извлечь текст из PDF.")

Приведение таблиц в лучший вид (Используется при индексации документов и приведении их в лучший вид)

In [81]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

def get_embedding_from_gigachain(text):
    # Определение контекста для обработки текста
    context = (
        "Во время предоставления представь что ты составляешь эмбединг, который потом будет прогоняться через вектор, ответ не должен превышать 500 символов"
        "Можешь переписать текст, так чтобы выделить в нем главную информацию и найди в этом "
        "фрагменте значения полей графиков и преобразуй их в таблицу по твоему усмотрению.\n"
        "Необходима таблица вида например:\n"
        "+-------------------+-------------------+\n"
        "|      Заголовок 1  |      Заголовок 2  |\n"
        "+-------------------+-------------------+\n"
        "|      Значение 1   |      Значение 2   |\n"
        "+-------------------+-------------------+\n"
        "|      Значение 3   |      Значение 4   |\n"
        "+-------------------+-------------------+\n"
        "или вида:\n"
        "+-------------------+-------------------+\n"
        "| Заголовок 1       | Заголовок 2       |\n"
        "+-------------------+-------------------+\n"
        "| Верхняя строка 1  | Верхняя строка 2  |\n"
        "+-------------------+-------------------+\n"
        "| Значение 1        | Значение 2        |\n"
        "| Значение 3        | Значение 4        |\n"
        "+-------------------+-------------------+\n"
        "Ну или любая другая таблица на твое усмотрение, лишь бы ты понял."
    )

    # Настраиваем шаблон запроса
    prompt_template = PromptTemplate(
        input_variables=["text", "context"],
        template="{context}\n\nТекст: {text}"
    )

    auth = "MTA4ZDM1OTQtODA5Yi00NDMzLTk3Y2ItZjNjNTZiODRkOGY0Ojk5YjI5MzUwLThkYzYtNGJhMy04ZGQ5LTk5MTYwYmNhYmRmMg=="

    llm = GigaChat(
        credentials=auth,
        model="GigaChat:latest",  # Убедитесь, что эта модель доступна
        verify_ssl_certs=False,
        profanity_check=False
    )

    # Создаем цепочку для выполнения запроса
    chain = LLMChain(
        llm=llm,
        prompt=prompt_template
    )

    # Выполняем запрос и обрабатываем результат
    try:
        result = chain.run({"text": text, "context": context})
        return result  # Результат должен включать таблицу или обработанный текст
    except Exception as e:
        print(f"Ошибка при выполнении запроса через GigaChain: {e}")
        return None

# text_to_process = "го роста до 2030 г. \
# Направление1 Целевые показатели\
# Рост объемов добычи Рост добычи в Норильском промышленном районе2:\
# К 2025 г.  \
# до 24‒26 млн т 2030 г.+  \
# до 32‒34 млн т\
# Модернизация  \
# перерабатывающих  \
# мощностейУвеличение производства металлов (к 2030 г.+)3:\
# Никель\
# на 25-35% Медь\
# на 25-35%Металлы платиновой группы\
# на 50-60%\
# 1  Более подробная информация о проектах по направлениям «Рост объемов..."
# result = get_embedding_from_gigachain(text_to_process)
# print("Результат:", result)




Результат: Для того чтобы преобразовать текст в таблицу, нужно выделить ключевую информацию и разделить ее на столбцы. Вот пример таблицы с заголовками "Направление" и "Целевые показатели":

```
+--------------+---------------------+
| Направление | Целевые показатели  |
+--------------+---------------------+
| Направление1 | Рост объемов добычи |
|             | к 2025 г.           |
|             | до 24‒26 млн т       |
|             | к 2030 г.            |
|             | + до 32‒34 млн т     |
| Направление2 | Модернизация         |
|             | перерабатывающих    |
|             | мощностей           |
|             | Увеличение           |
|             | производства металлов |
|             | к 2030 г.            |
|             | никеля на 25-35%     |
|             | меди на 25-35%       |
|             | металлов платиновой  |
|             | группы на 50-60%     |
+--------------+---------------------+
```

В этой таблице столбцы соответствуют названиям направлений и 

Извлечение DOCX формата

In [ ]:
from docx import Document
from langchain.schema import Document as LangchainDocument

def read_docx(file_path):
    """Извлекает текст из документа .docx"""
    doc = Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text.strip()

def split_text_into_fragments(text, fragment_length=400, overlap=100):
    """
    Разделяет текст на фрагменты с наложением.
    """
    start = 0
    fragments_with_metadata = []
    while start < len(text):
        end = min(start + fragment_length, len(text))
        fragment = text[start:end]
        fragments_with_metadata.append(
            LangchainDocument(page_content=fragment, metadata={})
        )
        start += fragment_length - overlap  # Двигаемся с учетом наложения
    return fragments_with_metadata

def process_docx_and_create_index(input_folder, output_index_file, fragment_length=400, overlap=100):
    """Обрабатывает все .docx файлы в папке и создает общий векторный индекс."""
    all_fragments = []
    docx_files = [f for f in os.listdir(input_folder) if f.endswith('.docx')]

    if not docx_files:
        print("Нет .docx файлов в указанной папке.")
        return

    for docx_file in docx_files:
        file_path = os.path.join(input_folder, docx_file)
        print(f"Обрабатывается файл: {docx_file}")
        text = read_docx(file_path)
        fragments = split_text_into_fragments(text, fragment_length=fragment_length, overlap=overlap)
        all_fragments.extend(fragments)

    # Параметры модели
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

    # Создание векторного хранилища
    vector_store = FAISS.from_documents(all_fragments, embedding)

    # Сохранение индекса
    vector_store.save_local(output_index_file)
    print(f"Индекс с метаданными сохранен в файл: {output_index_file}")

Индексация

In [88]:
def read_text_from_file(file_path):
    """Читает текст из файла."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_into_pages_and_fragments_with_metadata(text, file_path, fragment_length=400, overlap=100):
    """
    Разделяет текст на страницы и фрагменты с наложением.
    Добавляет метаданные с номерами страниц, именем файла и полным путем.
    """
    pages = text.split("Page ")
    pages = [page.strip() for page in pages if page.strip()]  # Убираем пустые строки

    fragments_with_metadata = []
    file_name = os.path.basename(file_path)  # Имя файла
    file_path_absolute = os.path.abspath(file_path)  # Абсолютный путь к файлу

    for page_number, page_content in enumerate(pages, start=1):
        start = 0
        while start < len(page_content):
            end = min(start + fragment_length, len(page_content))
            fragment = page_content[start:end]
            fragments_with_metadata.append(
                Document(
                    page_content=fragment,
                    metadata={
                        "page": page_number,
                        "file_name": file_name,
                        "file_path": file_path_absolute,  # Полный путь к файлу
                    }
                )
            )
            start += fragment_length - overlap  # Двигаемся с учетом наложения
    return fragments_with_metadata


def process_folder_and_create_index(input_folder, output_index_file, fragment_length=500, overlap=100):
    """Обрабатывает все текстовые файлы в папке и создает общий векторный индекс."""
    all_fragments = []
    txt_files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]

    if not txt_files:
        print("Нет текстовых файлов в указанной папке.")
        return

    for txt_file in txt_files:
        file_path = os.path.join(input_folder, txt_file)
        print(f"Обрабатывается файл: {file_path}")
        text = read_text_from_file(file_path)
        fragments = split_text_into_pages_and_fragments_with_metadata(
            text, file_path=file_path, fragment_length=fragment_length, overlap=overlap
        )
        all_fragments.extend(fragments)

    # Параметры модели
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

    # Создание векторного хранилища
    vector_store = FAISS.from_documents(all_fragments, embedding)

    # Сохранение индекса
    vector_store.save_local(output_index_file)
    print(f"Индекс с метаданными сохранен в файл: {output_index_file}")



input_folder = "extracted_texts"  # Папка с текстовыми файлами
output_index_file = "vector_index_with_metadata.faiss"  # Имя файла индекса
vector_store = process_folder_and_create_index(input_folder, output_index_file)

Обрабатывается файл: extracted_texts/Норникель про корп культуру.txt
Обрабатывается файл: extracted_texts/NN_CSO2021_RUS_03.03.2023.txt
Обрабатывается файл: extracted_texts/McKinsey_Next Big Arenas_2024 (213 pgs).txt
Обрабатывается файл: extracted_texts/ММК 2024.txt
Обрабатывается файл: extracted_texts/KPMG_Global Metals & Mining_2024 (48 pgs).txt
Обрабатывается файл: extracted_texts/2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.txt
Обрабатывается файл: extracted_texts/nn_cso_2023_rus.txt
Обрабатывается файл: extracted_texts/Kept_Золотодобывающие_компании_2024_25_стр.txt
Обрабатывается файл: extracted_texts/Росконгресс_Рынок_промышленных_роботов_в_мире_и_России_2024_16_стр.txt
Обрабатывается файл: extracted_texts/digital_production_5.txt
Обрабатывается файл: extracted_texts/K2Tex_x_TeДо_Российский_рынок_ИТ_2024_30_стр.txt
Обрабатывается файл: extracted_texts/nn_climate_change_report_rus.txt
Обрабатывается файл: extracted_texts/2_5366183129474161642.txt
Обрабатывается файл: extracte

Постобработка эмбедингов (используется для лучшего поиска и понимания какой контекст наиболее подходящий)

In [61]:
from torch.utils.data import DataLoader
import torch
from PIL import Image
import os
from colpali_engine.models import ColQwen2, ColQwen2Processor
import gc

# Загрузка модели и процессора для пост обработки нужных эмбедингов
model_name = "vidore/colqwen2-v0.1"
model = ColQwen2.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Используем половинную точность
    device_map="cuda:0",
).eval()
processor = ColQwen2Processor.from_pretrained(model_name)

# Функция для обработки изображений по батчам
def process_images_in_batches(model, processor, images, batch_size, query):

    # Путь к папке с изображениями
    image_folder = "output_images"
    images = [Image.open(os.path.join(image_folder, img)) for img in os.listdir(image_folder)]

    # Обработка текста
    batch_query = processor.process_queries([query]).to(model.device, dtype=torch.float16)
    embeddings = []
    dataloader = DataLoader(images, batch_size=batch_size, shuffle=False)

    with torch.no_grad():
        for batch in dataloader:
            batch_images = processor.process_images(batch).to(model.device, dtype=torch.float16)
            batch_embeddings = model(**batch_images)
            embeddings.append(batch_embeddings)

            # Очистка памяти
            del batch_images, batch_embeddings
            torch.cuda.empty_cache()
            gc.collect()

    return torch.cat(embeddings, dim=0)

# Вывод
# score


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

Финальный запрос в LLM модель для получения последнего ответа

In [94]:
# Загрузка FAISS векторных представлений
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
index_file = "vector_index_with_metadata.faiss"
vector_store = FAISS.load_local(index_file, embedding, allow_dangerous_deserialization=True)

embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Настройка модели GigaChat
auth = 'MTA4ZDM1OTQtODA5Yi00NDMzLTk3Y2ItZjNjNTZiODRkOGY0Ojk5YjI5MzUwLThkYzYtNGJhMy04ZGQ5LTk5MTYwYmNhYmRmMg=='

llm = GigaChat(
    credentials=auth,
    model='GigaChat-Pro',
    verify_ssl_certs=False,
    profanity_check=False
)

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Не надо задавать обратных вопросов!\
Если в контексте недостаточно информации то попробуй аккуратно соврать, чтобы не было понятно что есть ли на самом деле какая-то информация или нет\
Если в контексте ты не найдешь нужную информацию,\
то приведи какой-то ответ так как-будто он там есть и ОБЯЗАТЕЛЬНО скажи что\
"ниже представленны страницы PDF файлов вы можете попытаться найти нужную информацию там""\
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)
q1 = 'Мультипликатор свободного денежного потока в отношении 23 к 24 году'

# Запрос к цепочке
resp1 = retrieval_chain.invoke({'input': q1})

similar_docs = vector_store.similarity_search(q1, k=5)

print("Ответ модели:", resp1['answer'])
for doc in similar_docs[:5]:
    print(f"Чанк текста: {doc.page_content[:400]}...")
    print(f"Метаданные: {doc.metadata}")

Ответ модели: Мультипликатор свободного денежного потока в 2023 году по сравнению с 2024 годом составляет 6. Это означает, что показатель свободного денежного потока в 2023 году был в 6 раз выше, чем в 2024 году. Однако, поскольку вопрос касается 2023 и 2024 годов, ниже представлены страницы PDF файлов, где вы можете попытаться найти более подробную информацию.
Чанк текста: тем балансиро -
вания денежных потоков по выручке, 
выраженных в основном в долла -
рах США, и по обязательствам, выра -
женным в рублях.
Балансовая стоимость монетар -
ных активов и обязательств, помимо 
валютно-процентных свопов, дено -
минированных в  иностранной 
валюте, отличной от функциональной 
валюты отдельных компаний Группы 
по состоянию на 31 декабря 2023, 
2022 и 2021 годов представле...
Метаданные: {'page': 150, 'file_name': 'Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god.txt', 'file_path': '/content/extracted_texts/Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god.txt'}
Чанк текста: к дол -
л

Надо для того, чтобы вытащить страницы PDF

In [26]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 2s (117 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 123632 

Код для извлечения нужных страниц PDF файлов

In [91]:
def extract_page_as_image(pdf_path, page_number):
    """
    Извлекает указанную страницу из PDF в виде изображения.
    """
    images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)
    return images[0]  # Возвращаем единственное изображение, соответствующее странице

def save_image(image, output_folder, filename):
    """
    Сохраняет изображение в указанную папку.
    """
    os.makedirs(output_folder, exist_ok=True)  # Создаем папку, если её нет
    output_path = os.path.join(output_folder, filename)
    image.save(output_path, format='PNG')  # Сохраняем изображение в формате PNG
    return output_path

def extract_and_save_page_image(pdf_path, page_number, output_folder):
    """
    Извлекает указанную страницу из PDF и сохраняет её как изображение.

    :param pdf_path: Путь к PDF-файлу
    :param page_number: Номер страницы для извлечения
    :param output_folder: Папка для сохранения изображений
    :return: Путь к сохраненному изображению
    """
    try:
        # Извлечение страницы как изображения
        page_image = extract_page_as_image(pdf_path, page_number)

        # Формирование имени файла
        filename = f"page_{page_number}.png"

        # Сохранение изображения
        output_path = save_image(page_image, output_folder, filename)
        print(f"Страница {page_number} из {pdf_path} сохранена: {output_path}")
        return output_path

    except Exception as e:
        print(f"Ошибка при обработке страницы {page_number} из {pdf_path}: {e}")
        return None

output_folder = 'output_images'

for doc in similar_docs[:5]:
    extract_and_save_page_image(f"pdfs/{doc.metadata['file_name'][:-3]}pdf", doc.metadata['page'], output_folder)


Страница 103 из pdfs/NN_CSO2021_RUS_03.03.2023.pdf сохранена: output_images/page_103.png
Страница 161 из pdfs/NN_AR_2021_Book_RUS_26.09.22.pdf сохранена: output_images/page_161.png
Страница 228 из pdfs/2_5366183129474161642.pdf сохранена: output_images/page_228.png
Страница 33 из pdfs/nn_climate_change_report_rus.pdf сохранена: output_images/page_33.png
Страница 88 из pdfs/NN_AR_2021_Book_RUS_26.09.22.pdf сохранена: output_images/page_88.png


Количество чанков в индексации

In [67]:
chunk_count = vector_store.index.ntotal
chunk_count

20390